In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
import requests

api_file = open("apikey.txt", "r")
api_key = api_file.read()
api_file.close()

url_geocode = "https://maps.googleapis.com/maps/api/geocode/json?"
url_directions = "https://maps.googleapis.com/maps/api/directions/json?"

north_bound = 51.438850
south_bound = 51.404455
left_bound = 6.865338
right_bound = 6.906226

In [ ]:
nc = 125
nr = 4 #

locations = [[(north_bound + south_bound) / 2, (left_bound + right_bound) / 2]]

import random
for i in range(1, nc + 1):
    locations.append([random.uniform(south_bound, north_bound), random.uniform(left_bound, right_bound)])
    
locations.append([north_bound * 0.25 + south_bound * 0.75, left_bound * 0.25 + right_bound * 0.75])
locations.append([north_bound * 0.75 + south_bound * 0.25, left_bound * 0.25 + right_bound * 0.75])
locations.append([north_bound * 0.25 + south_bound * 0.75, left_bound * 0.75 + right_bound * 0.25])
locations.append([north_bound * 0.75 + south_bound * 0.25, left_bound * 0.75 + right_bound * 0.25])

print(locations)

In [ ]:
def find_nearby_address(index):
    lat = locations[index][0]
    lng = locations[index][1]
    response = requests.get(url_geocode + 'latlng=' + str(lat) + ',' + str(lng) + '&key=' + api_key)
    dic = response.json()
    if dic['results'][0]['address_components'][0]['types'] == ['street_number']:
        address = dic['results'][0]['formatted_address']
        lat_new = dic['results'][0]['geometry']['location']['lat']
        lng_new = dic['results'][0]['geometry']['location']['lng']
        return (lat_new, lng_new, address)
    search_radius = 0.001
    while True:
        count = 0
        while count < 20:
            lat_new = lat + random.uniform(-search_radius, search_radius)
            lng_new = lng + random.uniform(-search_radius, search_radius)
            response = requests.get(url_geocode + 'latlng=' + str(lat_new) + ',' + str(lng_new) + '&key=' + api_key)
            dic = response.json()
            if dic['results'][0]['address_components'][0]['types'] == ['street_number']:
                address = dic['results'][0]['formatted_address']
                lat_new = dic['results'][0]['geometry']['location']['lat']
                lng_new = dic['results'][0]['geometry']['location']['lng']
                return (lat_new, lng_new, address)
            count += 1
        search_radius += 0.001

In [ ]:
locations_new = []
addresses = []

(lat, lng, address) = find_nearby_address(0)
locations_new.append([lat, lng])
addresses.append(address)

for i in range(1, nc + 1):
    lat = locations[i][0]
    lng = locations[i][1]
    response = requests.get(url_geocode + 'latlng=' + str(lat) + ',' + str(lng) + '&key=' + api_key)
    dic = response.json()
    while dic['status'] == 'ZERO_RESULTS' or dic['results'][0]['address_components'][0]['types'] != ['street_number']:
        lat = random.uniform(south_bound, north_bound)
        lng = random.uniform(left_bound, right_bound)
        response = requests.get(url_geocode + 'latlng=' + str(lat) + ',' + str(lng) + '&key=' + api_key)
        dic = response.json()
    lat = dic['results'][0]['geometry']['location']['lat']
    lng = dic['results'][0]['geometry']['location']['lng']
    locations_new.append([lat,lng])
    addresses.append(dic['results'][0]['formatted_address'])

for i in range(nc + 1, nc + nr + 1):
    (lat, lng, address) = find_nearby_address(i)
    locations_new.append([lat, lng])
    addresses.append(address)

In [ ]:
print(locations_new)
print(addresses)

In [ ]:
distances = {}
routes = {}
count = 0
total = len(addresses) * (len(addresses) - 1)

for i in range(0, len(addresses)):
    for j in range(i + 1, len(addresses)):
        origin = addresses[i]
        destination = addresses[j]
        response = requests.get(url_directions + "origin=" + origin + "&destination=" + destination+ "&key=" + api_key)
        route = response.json()
        routes[(i, j)] = route
        distance = route["routes"][0]["legs"][0]["distance"]["value"]
        distances[(i, j)] = distance / 1000
        
        routes[(j, i)] = route
        distances[(j, i)] = distance / 1000
        
        '''
        origin = addresses[j]
        destination = addresses[i]
        response = requests.get(url_directions + "origin=" + origin + "&destination=" + destination+ "&key=" + api_key)
        route = response.json()
        routes[(j, i)] = route
        distance = route["routes"][0]["legs"][0]["distance"]["value"]
        distances[(j, i)] = distance / 1000
        '''
        
        count += 2
        print('%3.2f percent complete' %(count / total * 100), end = '\r')
        

In [ ]:
for i in range(0, nc + nr + 1):
    distances[(i, i)] = 0

In [ ]:
print(distances)

In [ ]:
print(routes.keys())

In [ ]:
from tkinter import filedialog
from tkinter import Tk
import os

root = Tk()
path = filedialog.asksaveasfilename(initialdir = os.getcwd(), title = 'Please select a file:')

root.destroy()

print(path)

In [ ]:
service_time_customer = 0.1666
service_time_rs = 1
speed = 40
battery_consum_rate = 1
battery_full_level = 15
maximum_time = 1

import pickle
f = open(path,'wb')
pickle.dump(locations_new, f)
pickle.dump(addresses, f)
pickle.dump(distances, f)
pickle.dump(routes, f)

pickle.dump(north_bound, f)
pickle.dump(south_bound, f)
pickle.dump(left_bound, f)
pickle.dump(right_bound, f)
pickle.dump(nc, f)
pickle.dump(nr, f)
pickle.dump(service_time_customer, f)
pickle.dump(service_time_rs, f)
pickle.dump(speed, f)
pickle.dump(battery_consum_rate, f)
pickle.dump(battery_full_level, f)
pickle.dump(maximum_time, f)
f.close()

In [ ]:
lats, lngs = zip(*locations_new)

import gmplot

gmap = gmplot.GoogleMapPlotter((north_bound + south_bound) / 2, (left_bound + right_bound) / 2, 13, apikey = api_key)

gmap.marker(north_bound, left_bound, color = 'black')
gmap.marker(south_bound, left_bound, color = 'black')
gmap.marker(north_bound, right_bound, color = 'black')
gmap.marker(south_bound, right_bound, color = 'black')

gmap.marker(lats[0], lngs[0], color = 'red')

gmap.scatter(lats[1: (nc + 1)], lngs[1: (nc + 1)], color = 'blue', size = 100, marker = False)

for i in range(0, nr):
    
    gmap.marker(lats[1+ nc + i], lngs[1+ nc + i], color = 'gold')
    
url = 'map_' + path[path.rindex('/') + 1:] + '.html'

gmap.draw(url)

from IPython.display import IFrame

IFrame(url, width = 500, height = 500)